In [10]:
import logging

import nshutils as nu
import rich

logging.basicConfig(level=logging.DEBUG)

nu.pretty()

INFO:root:Failed to import `lovely_tensors`. Ignoring pretty PyTorch tensor formatting
INFO:root:Failed to import `lovely_numpy`. Ignoring pretty numpy array formatting


In [5]:
from __future__ import annotations

import os
from pathlib import Path

import mattertune.configs as MC
from mattertune import MatterTuner

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


def hparams():
    hparams = MC.MatterTunerConfig.draft()

    # Model hparams 
    hparams.model = MC.MatterSimBackboneConfig.draft()
    hparams.model.graph_convertor = MC.MatterSimGraphConvertorConfig.draft()

    hparams.model.pretrained_model = "MatterSim-v1.0.0-5M"
    hparams.model.ignore_gpu_batch_transform_error = True
    hparams.model.optimizer = MC.AdamWConfig(lr=1.0e-4)

    hparams.model.properties = []
    energy = MC.EnergyPropertyConfig(loss=MC.MAELossConfig())
    hparams.model.properties.append(energy)
    forces = MC.ForcesPropertyConfig(loss=MC.MAELossConfig(), conservative=True)
    hparams.model.properties.append(forces)
    stresses = MC.StressesPropertyConfig(loss=MC.MAELossConfig(), conservative=True)
    hparams.model.properties.append(stresses)

    # Data hparams
    hparams.data = MC.AutoSplitDataModuleConfig.draft()
    hparams.data.dataset = MC.XYZDatasetConfig.draft()
    hparams.data.dataset.src = Path("../examples/water-thermodynamics/data/water_ef.xyz")
    hparams.data.train_split = 0.03
    hparams.data.batch_size = 2

    # Trainer hparams
    hparams.trainer.additional_trainer_kwargs = {"fast_dev_run": True}

    hparams = hparams.finalize()
    rich.print(hparams)
    return hparams


hp = hparams()
tune_output = MatterTuner(hp).tune()
model, trainer = tune_output.model, tune_output.trainer

MatterTunerConfig(
    data=AutoSplitDataModuleConfig(
        batch_size=2,
        num_workers='auto',
        pin_memory=True,
        dataset=XYZDatasetConfig(type='xyz', src=PosixPath('../examples/water-thermodynamics/data/water_ef.xyz')),
        train_split=0.03,
        validation_split='auto',
        shuffle=True,
        shuffle_seed=42
    ),
    model=MatterSimBackboneConfig(
        properties=[
            EnergyPropertyConfig(
                name='energy',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='energy'
            ),
            ForcesPropertyConfig(
                name='forces',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='forces',
                conservative=True
            ),
            StressesPropertyConfig(
                name='stresses',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='stresses',
                conservative=True
            )
        ],
        optimizer=AdamWConfig(
            name='AdamW',
            lr=0.0001,
            eps=1e-08,
            betas=(0.9, 0.999),
            weight_decay=0.01,
            amsgrad=False
        ),
        lr_scheduler=None,
        ignore_gpu_batch_transform_error=True,
        normalizers={},
        name='mattersim',
        pretrained_model='MatterSim-v1.0.0-5M',
        model_type='m3gnet',
        graph_convertor=MatterSimGraphConvertorConfig(
            twobody_cutoff=5.0,
            has_threebody=True,
            threebody_cutoff=4.0
        )
    ),
    trainer=TrainerConfig(
        accelerator='auto',
        strategy='auto',
        num_nodes=1,
        devices='auto',
        precision='32-true',
        deterministic=None,
        max_epochs=None,
        min_epochs=None,
        max_steps=-1,
        min_steps=None,
        max_time=None,
        val_check_interval=None,
        check_val_every_n_epoch=1,
        log_every_n_steps=None,
        gradient_clip_val=None,
        gradient_clip_algorithm=None,
        checkpoint=None,
        early_stopping=None,
        loggers='default',
        additional_trainer_kwargs={'fast_dev_run': True}
    )
)

2024-12-16 12:36:10.897 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model


INFO:mattertune.main:The model requires inference_mode to be disabled. Setting inference_mode=False.
INFO: Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning.pytorch.utilities.rank_zero:Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, usin

Epoch 0: 100%|██████████| 1/1 [00:08<00:00,  0.12it/s]

INFO: `Trainer.fit` stopped: `max_steps=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:08<00:00,  0.12it/s]


In [6]:
property_predictor = model.property_predictor()
print(property_predictor)

calculator = model.ase_calculator()
print(calculator)

In [7]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

print(property_predictor.predict([atoms], model.hparams.properties))

Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])


INFO:mattertune.wrappers.property_predictor:The model requires inference_mode to be disabled. Setting inference_mode=False.
INFO: You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
INFO:lightning.pytorch.utilities.rank_zero:You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing 

[{'total_energy': tensor([-11.2461]), 'forces': tensor([[-0.0000, -1.0471, -3.0894],
        [-0.0000,  3.0241,  0.0653],
        [-0.0000, -1.9770,  3.0241]]), 'stresses': tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.3167, -0.4845],
         [ 0.0000, -0.4845, -0.0105]]]), 'energy': tensor([-11.2461])}]


In [8]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

# Set the calculator
atoms.calc = calculator

# Calculate the energy
energy = atoms.get_potential_energy()
print(energy)

INFO:mattertune.wrappers.property_predictor:The model requires inference_mode to be disabled. Setting inference_mode=False.
INFO: You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
INFO:lightning.pytorch.utilities.rank_zero:You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing 

Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])
-11.246139526367188


In [9]:
property_predictor = model.property_predictor()
atoms_1 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms_2 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms = [atoms_1, atoms_2]
predictions = property_predictor.predict(atoms, ["energy", "forces"])
print("ase.Atoms 1 energy:", predictions[0]["energy"])
print("ase.Atoms 1 forces:", predictions[0]["forces"])
print("ase.Atoms 2 energy:", predictions[1]["energy"])
print("ase.Atoms 2 forces:", predictions[1]["forces"])

INFO:mattertune.wrappers.property_predictor:The model requires inference_mode to be disabled. Setting inference_mode=False.
INFO: You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing the Trainer overhead. Specifically, the following features are deactivated:
 - Checkpointing: `Trainer(enable_checkpointing=True)`
 - Progress bar: `Trainer(enable_progress_bar=True)`
 - Model summary: `Trainer(enable_model_summary=True)`
 - Logging: `Trainer(logger=True)`, `Trainer(log_every_n_steps>0)`, `LightningModule.log(...)`, `LightningModule.log_dict(...)`
 - Sanity checking: `Trainer(num_sanity_val_steps>0)`
 - Development run: `Trainer(fast_dev_run=True)`
 - Anomaly detection: `Trainer(detect_anomaly=True)`
 - Profiling: `Trainer(profiler=...)`
INFO:lightning.pytorch.utilities.rank_zero:You are running in `Trainer(barebones=True)` mode. All features that may impact raw speed have been disabled to facilitate analyzing 

ase.Atoms 1 energy: tensor([-11.2461])
ase.Atoms 1 forces: tensor([[-0.0000, -1.0471, -3.0894],
        [-0.0000,  3.0241,  0.0653],
        [-0.0000, -1.9770,  3.0241]])
ase.Atoms 2 energy: tensor([-11.2461])
ase.Atoms 2 forces: tensor([[-0.0000, -1.0471, -3.0894],
        [-0.0000,  3.0241,  0.0653],
        [-0.0000, -1.9770,  3.0241]])
